In [1]:
import gym
from gym import spaces
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque, namedtuple
from torch.optim.lr_scheduler import ExponentialLR


# Hyperparameters
MAX_VALUE = 32
MAX_LIST_LENGTH = 16
LIST_LENGTH = 7
MAX_STEPS = 80  #LIST_LENGTH * LIST_LENGTH * LIST_LENGTH / 4
HIDDEN_SIZE = 128
BATCH_SIZE = 64
GAMMA = 0.99
EPS_START = 0.2
EPS_END = 0.05
EPS_DECAY = 2000
TARGET_UPDATE = 10
LR = 1e-3
LR_DECAY = 0.9995
NUM_EPISODES = 40000
NUM_STEPS_IN_EPISODE = 10000

SUCCESS_REWARD = 10.0
SWAP_REWARD = 0.1
STEP_PENALTY = 0.01


# Define the custom environment
class SortingEnv(gym.Env):
    def __init__(self, list_length=LIST_LENGTH, max_list_length=MAX_LIST_LENGTH):
        super(SortingEnv, self).__init__()
        self.list_length = list_length
        self.max_list_length = max_list_length
        self.action_space = spaces.Discrete(4)  # 0: Move Left, 1: Move Right, 2: Compare, 3: Swap
        # Observation space: the list and cursor position
        # Values are normalized between 0 and 1
        self.observation_space = spaces.Box(
            low=0.0, high=1.0, shape=(self.max_list_length + 1,), dtype=np.float32
        )
        self.reset()

    def reset(self):
        self.list = np.random.randint(0, MAX_VALUE, size=self.list_length)
        self.cursor = 0
        self.done = False
        self.steps = 0
        return self._get_obs()

    def _get_obs(self):
        # Normalize list values and cursor position
        normalized_list = np.concatenate((self.list / MAX_VALUE, np.zeros(self.max_list_length - self.list_length)))
        normalized_cursor = np.array([self.cursor / (self.list_length - 1)])
        obs = np.concatenate((normalized_list, normalized_cursor))
        return obs.astype(np.float32)

    def step(self, action):
        reward = 0
        self.steps += 1
        # Action definitions
        if action == 0:  # Move cursor left
            if self.cursor > 0:
                self.cursor -= 1
            else:
                reward -= 1  # Penalty for invalid move
        elif action == 1:  # Move cursor right
            if self.cursor < self.list_length - 2:
                self.cursor += 1
            else:
                reward -= 1  # Penalty for invalid move
        elif action == 2:  # Compare (no effect in this setup)
            pass
        elif action == 3:  # Swap
            # Give a little reward if swapping in the right way, and vice versa
            if self.list[self.cursor] > self.list[self.cursor + 1]:
                reward += SWAP_REWARD
            if self.list[self.cursor] < self.list[self.cursor + 1]:
                reward -= SWAP_REWARD
            # Swap elements at cursor and cursor + 1
            self.list[self.cursor], self.list[self.cursor + 1] = (
                self.list[self.cursor + 1],
                self.list[self.cursor],
            )
        else:
            reward -= 1  # Penalty for invalid action

        # Check if the list is sorted
        if np.all(self.list[:-1] <= self.list[1:]):
            reward += SUCCESS_REWARD  # Reward for sorting the list
            self.done = True
        else:
            reward -= STEP_PENALTY  # Small penalty to encourage efficiency

        # Limit the number of steps to prevent infinite episodes
        max_steps = MAX_STEPS #self.list_length * self.list_length
        if self.steps >= max_steps: 
            #print("MAX_STEP =", self.list_length * self.list_length)
            self.done = True

        obs = self._get_obs()
        info = {}
        return obs, reward, self.done, info

# Define the DQN neural network
class DQN(nn.Module):
    def __init__(self, observation_space, action_space):
        super(DQN, self).__init__()
        print("observation_space size =", observation_space.shape[0])
        self.fc = nn.Sequential(
            nn.Linear(observation_space.shape[0], HIDDEN_SIZE),
            nn.ReLU(),
            nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE),
            nn.ReLU(),
            nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE),
            nn.ReLU(),
            nn.Linear(HIDDEN_SIZE, action_space.n),
        )

    def forward(self, x):
        return self.fc(x)

# Replay memory to store transitions
class ReplayMemory:
    def __init__(self, capacity=20000):
        self.memory = deque(maxlen=capacity)

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        return Transition(*zip(*batch))

    def __len__(self):
        return len(self.memory)
    
# Replay memory to store transitions
class BipartiteReplayMemory:
    def __init__(self, capacity=20000):
        self.pos_memory = deque(maxlen=capacity//2)
        self.neg_memory = deque(maxlen=capacity//2)

    def push(self, *args):
        transition = Transition(*args)
        reward = transition.reward.cpu().numpy()[0]
        if reward > SWAP_REWARD * 2:
            self.pos_memory.append(transition)
        else:
            self.neg_memory.append(transition)

    def sample(self, batch_size):
        batch = random.sample(self.pos_memory, min(batch_size // 2, len(self.pos_memory) // 4))
        batch.extend(random.sample(self.neg_memory, batch_size - len(batch)))
        return Transition(*zip(*batch))

    def __len__(self):
        return len(self.pos_memory) + len(self.neg_memory)
    
# Replay memory to store transitions
class PrioritizedReplayMemory:
    def __init__(self, capacity=1000000):
        self.capacity = capacity
        self.memory = deque(maxlen=capacity)
        self.priorities = deque(maxlen=capacity)

    def push(self, *args):
        transition = Transition(*args)
        self.memory.append(transition)
        self.priorities.append(transition.reward.cpu().numpy()[0])

    def sample(self, batch_size):
        #batch = random.sample(self.memory, batch_size)
        #return Transition(*zip(*batch))

        priorities = np.array(self.priorities)
        indices = np.arange(len(self.memory))

        # Separate indices into positive and zero priorities
        positive_indices = indices[priorities > 0]
        negative_indices = indices[priorities <= 0]

        # Determine the number of positive samples
        half_batch = batch_size // 2
        num_positive = len(positive_indices)

        if num_positive >= half_batch:
            # Sample half of batch_size from positive indices
            positive_sample_indices = np.random.choice(positive_indices, half_batch, replace=False)
        else:
            # Use all positive indices
            positive_sample_indices = positive_indices

        num_positive_samples = len(positive_sample_indices)

        # Sample the remaining from zero priorities
        num_negative_samples = batch_size - num_positive_samples

        if len(negative_indices) >= num_negative_samples:
            negative_sample_indices = np.random.choice(negative_indices, num_negative_samples, replace=False)
        else:
            # Not enough zero priority samples; sample with replacement
            negative_sample_indices = np.random.choice(negative_indices, num_negative_samples, replace=True)

        # Combine positive and zero priority indices
        total_sample_indices = np.concatenate((positive_sample_indices, negative_sample_indices))

        # Shuffle the indices to mix positive and zero priority samples
        np.random.shuffle(total_sample_indices)

        # Retrieve transitions
        transitions = [self.memory[idx] for idx in total_sample_indices]

        # Create batch
        batch = Transition(*zip(*transitions))
        return batch

    def __len__(self):
        return len(self.memory)

    def num_positives(self):
        return sum(1 for x in self.priorities if x > 0)
    

# Initialize environment, networks, optimizer, and memory
env = SortingEnv(list_length=LIST_LENGTH)
n_actions = env.action_space.n
state_size = env.observation_space.shape[0]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

policy_net = DQN(env.observation_space, env.action_space).to(device)
#policy_net.load_state_dict(torch.load('datasets/dqn_4action_numeric/len6_maxlen16_3layer_hd128_20kepoch_accr8960_policynet.pth'))
target_net = DQN(env.observation_space, env.action_space).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=LR)
scheduler = ExponentialLR(optimizer, gamma=LR_DECAY)

memory = BipartiteReplayMemory()
steps_done = 0
steps_positive = 0

Transition = namedtuple("Transition", ("state", "action", "next_state", "reward"))


# Epsilon-greedy action selection
def select_action(state):
    global steps_done
    global steps_positive
    steps_done += 1
    eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * min(steps_done, 20 * steps_positive) / EPS_DECAY)
    if random.random() < eps_threshold:
        return random.randrange(n_actions)
    else:
        with torch.no_grad():
            state = state.unsqueeze(0).to(device)  # Add batch dimension
            return policy_net(state).argmax(dim=1).item()

# Optimize the model
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*transitions)
    # Create masks and tensors
    non_final_mask = torch.tensor(
        tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool
    )
    non_final_next_states = torch.stack([s for s in batch.next_state if s is not None])
    state_batch = torch.stack(batch.state).to(device)
    action_batch = torch.cat(batch.action).view(-1, 1).to(device)
    reward_batch = torch.cat(batch.reward).to(device)
    
    # Compute Q(s_t, a)
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    
    # Compute V(s_{t+1})
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    
    # Compute expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    # Compute loss
    loss = nn.functional.smooth_l1_loss(state_action_values.squeeze(), expected_state_action_values)
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


# Main training loop
for i_episode in range(NUM_EPISODES):
    global steps_positive
    state = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device)
    total_reward = 0
    for t in range(NUM_STEPS_IN_EPISODE):
        action = select_action(state)
        action_tensor = torch.tensor([action], dtype=torch.long, device=device)
        next_state, reward, done, _ = env.step(action)
        if reward > 0:
            steps_positive += 1
            #print("steps_positive:", steps_positive)
        total_reward += reward
        reward = torch.tensor([reward], dtype=torch.float32, device=device)
        if not done:
            next_state = torch.tensor(next_state, dtype=torch.float32, device=device)
        else:
            next_state = None
        memory.push(state, action_tensor, next_state, reward)
        state = next_state
        optimize_model()
        if done:
            print(f"Episode {i_episode}, Total Reward: {total_reward}") #, Memory #positives: {memory.num_positives()}")
            break
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
    scheduler.step()

# Save the trained model
torch.save(policy_net.state_dict(), "datasets/sorting_dqn_model.pth")


Using device: cuda
observation_space size = 17
observation_space size = 17
Episode 0, Total Reward: -55.79999999999994
Episode 1, Total Reward: -6.099999999999986
Episode 2, Total Reward: -0.9000000000000007
Episode 3, Total Reward: -0.8000000000000005
Episode 4, Total Reward: -1.7000000000000006
Episode 5, Total Reward: -0.8000000000000004
Episode 6, Total Reward: -1.6000000000000003
Episode 7, Total Reward: -2.499999999999986
Episode 8, Total Reward: -1.9000000000000006
Episode 9, Total Reward: -0.29999999999999993
Episode 10, Total Reward: -1.6000000000000005
Episode 11, Total Reward: -0.8000000000000005
Episode 12, Total Reward: -0.7000000000000004
Episode 13, Total Reward: -1.9000000000000006
Episode 14, Total Reward: -4.699999999999999
Episode 15, Total Reward: -2.8
Episode 16, Total Reward: -0.5999999999999999
Episode 17, Total Reward: -2.8
Episode 18, Total Reward: -1.5000000000000002
Episode 19, Total Reward: -1.6999999999999997
Episode 20, Total Reward: -0.6
Episode 21, Total

Episode 180, Total Reward: -0.5000000000000002
Episode 181, Total Reward: -2.299999999999988
Episode 182, Total Reward: -0.09999999999999966
Episode 183, Total Reward: -0.3
Episode 184, Total Reward: -0.4000000000000002
Episode 185, Total Reward: -0.6000000000000003
Episode 186, Total Reward: -0.5999999999999998
Episode 187, Total Reward: -0.40000000000000013
Episode 188, Total Reward: 0.10000000000000007
Episode 189, Total Reward: 10.139999999999999
Episode 190, Total Reward: -1.5000000000000007
Episode 191, Total Reward: -0.7000000000000005
Episode 192, Total Reward: -1.8000000000000005
Episode 193, Total Reward: -0.8000000000000005
Episode 194, Total Reward: -0.7999999999999998
Episode 195, Total Reward: -0.5000000000000003
Episode 196, Total Reward: -0.8000000000000005
Episode 197, Total Reward: -2.8000000000000003
Episode 198, Total Reward: -1.7000000000000004
Episode 199, Total Reward: -0.8999999999999999
Episode 200, Total Reward: -0.6000000000000003
Episode 201, Total Reward: -

Episode 360, Total Reward: -0.19999999999999976
Episode 361, Total Reward: -0.7
Episode 362, Total Reward: -0.40000000000000013
Episode 363, Total Reward: -0.4000000000000003
Episode 364, Total Reward: -0.7999999999999999
Episode 365, Total Reward: -1.9
Episode 366, Total Reward: -0.5000000000000002
Episode 367, Total Reward: -0.6000000000000003
Episode 368, Total Reward: -0.4999999999999994
Episode 369, Total Reward: -0.8000000000000005
Episode 370, Total Reward: -0.7
Episode 371, Total Reward: -0.7000000000000004
Episode 372, Total Reward: -0.09999999999999953
Episode 373, Total Reward: -0.8000000000000005
Episode 374, Total Reward: -1.5000000000000007
Episode 375, Total Reward: -1.8000000000000005
Episode 376, Total Reward: -0.9000000000000001
Episode 377, Total Reward: -0.4000000000000001
Episode 378, Total Reward: -0.7000000000000003
Episode 379, Total Reward: -0.9000000000000002
Episode 380, Total Reward: -0.4999999999999996
Episode 381, Total Reward: -0.5999999999999995
Episode 

Episode 540, Total Reward: -0.3999999999999996
Episode 541, Total Reward: -1.4000000000000006
Episode 542, Total Reward: -1.7000000000000006
Episode 543, Total Reward: -1.9000000000000006
Episode 544, Total Reward: -0.8000000000000004
Episode 545, Total Reward: -0.3999999999999998
Episode 546, Total Reward: -1.3
Episode 547, Total Reward: -0.9000000000000006
Episode 548, Total Reward: -0.9000000000000005
Episode 549, Total Reward: -0.4999999999999996
Episode 550, Total Reward: -0.40000000000000024
Episode 551, Total Reward: -0.6999999999999997
Episode 552, Total Reward: 0.1000000000000003
Episode 553, Total Reward: -26.40000000000003
Episode 554, Total Reward: -0.7000000000000004
Episode 555, Total Reward: -0.4000000000000003
Episode 556, Total Reward: -0.8
Episode 557, Total Reward: -1.7000000000000004
Episode 558, Total Reward: -1.8000000000000007
Episode 559, Total Reward: -0.7000000000000004
Episode 560, Total Reward: -0.5000000000000003
Episode 561, Total Reward: -2.69999999999999

Episode 719, Total Reward: -0.6000000000000004
Episode 720, Total Reward: -0.6999999999999997
Episode 721, Total Reward: -1.5999999999999996
Episode 722, Total Reward: -0.6000000000000001
Episode 723, Total Reward: -0.6000000000000003
Episode 724, Total Reward: -0.10000000000000007
Episode 725, Total Reward: -0.6000000000000001
Episode 726, Total Reward: -0.7000000000000004
Episode 727, Total Reward: -0.7000000000000004
Episode 728, Total Reward: -0.30000000000000004
Episode 729, Total Reward: -1.7000000000000006
Episode 730, Total Reward: -0.5
Episode 731, Total Reward: -0.5999999999999998
Episode 732, Total Reward: -0.9000000000000007
Episode 733, Total Reward: -0.6000000000000003
Episode 734, Total Reward: -0.4000000000000003
Episode 735, Total Reward: -0.7000000000000004
Episode 736, Total Reward: -1.4000000000000001
Episode 737, Total Reward: -0.30000000000000004
Episode 738, Total Reward: -0.4000000000000003
Episode 739, Total Reward: -2.699999999999995
Episode 740, Total Reward:

...

Episode 39264, Total Reward: -0.39999999999999925
Episode 39265, Total Reward: -0.49999999999999967
Episode 39266, Total Reward: -0.6000000000000003
Episode 39267, Total Reward: -0.7000000000000004
Episode 39268, Total Reward: -0.5000000000000003
Episode 39269, Total Reward: -0.7999999999999998
Episode 39270, Total Reward: -0.8999999999999998
Episode 39271, Total Reward: -0.9
Episode 39272, Total Reward: -0.7000000000000004
Episode 39273, Total Reward: -0.5999999999999995
Episode 39274, Total Reward: -0.6999999999999996
Episode 39275, Total Reward: -0.5000000000000002
Episode 39276, Total Reward: -0.7999999999999999
Episode 39277, Total Reward: -0.5999999999999991
Episode 39278, Total Reward: -0.6
Episode 39279, Total Reward: -0.8999999999999999
Episode 39280, Total Reward: -0.8000000000000006
Episode 39281, Total Reward: -0.9000000000000006
Episode 39282, Total Reward: -0.8999999999999998
Episode 39283, Total Reward: -1.8000000000000005
Episode 39284, Total Reward: -0.8999999999999998

Episode 39437, Total Reward: -0.5999999999999993
Episode 39438, Total Reward: -2.799999999999998
Episode 39439, Total Reward: -2.699999999999988
Episode 39440, Total Reward: -0.7
Episode 39441, Total Reward: -0.9000000000000005
Episode 39442, Total Reward: -0.4999999999999993
Episode 39443, Total Reward: -0.8999999999999998
Episode 39444, Total Reward: -0.7999999999999997
Episode 39445, Total Reward: -0.5999999999999999
Episode 39446, Total Reward: -0.6999999999999995
Episode 39447, Total Reward: -0.6000000000000004
Episode 39448, Total Reward: -0.7999999999999998
Episode 39449, Total Reward: -0.6999999999999997
Episode 39450, Total Reward: -0.6000000000000003
Episode 39451, Total Reward: -0.6999999999999997
Episode 39452, Total Reward: -0.5999999999999996
Episode 39453, Total Reward: -0.49999999999999933
Episode 39454, Total Reward: -0.7999999999999999
Episode 39455, Total Reward: -0.6000000000000003
Episode 39456, Total Reward: -0.9000000000000006
Episode 39457, Total Reward: -1.4000

Episode 39610, Total Reward: -0.6999999999999995
Episode 39611, Total Reward: -0.9999999999999998
Episode 39612, Total Reward: -0.7999999999999997
Episode 39613, Total Reward: -0.6
Episode 39614, Total Reward: -0.8000000000000005
Episode 39615, Total Reward: -0.6999999999999998
Episode 39616, Total Reward: -0.6999999999999995
Episode 39617, Total Reward: -0.5999999999999996
Episode 39618, Total Reward: -0.7999999999999996
Episode 39619, Total Reward: -0.7000000000000001
Episode 39620, Total Reward: -0.6999999999999997
Episode 39621, Total Reward: -1.8000000000000005
Episode 39622, Total Reward: -0.8000000000000005
Episode 39623, Total Reward: -0.5000000000000002
Episode 39624, Total Reward: -0.5000000000000002
Episode 39625, Total Reward: -0.6000000000000003
Episode 39626, Total Reward: -0.8000000000000005
Episode 39627, Total Reward: -2.7999999999999936
Episode 39628, Total Reward: -0.8999999999999998
Episode 39629, Total Reward: -0.7
Episode 39630, Total Reward: -0.5999999999999998
E

Episode 39782, Total Reward: -0.3999999999999992
Episode 39783, Total Reward: -0.7999999999999997
Episode 39784, Total Reward: -0.5999999999999992
Episode 39785, Total Reward: -0.6000000000000003
Episode 39786, Total Reward: -0.8999999999999999
Episode 39787, Total Reward: -0.7000000000000004
Episode 39788, Total Reward: -1.7000000000000006
Episode 39789, Total Reward: -0.3999999999999997
Episode 39790, Total Reward: -0.7000000000000001
Episode 39791, Total Reward: -1.7000000000000004
Episode 39792, Total Reward: -0.7999999999999996
Episode 39793, Total Reward: -0.8999999999999998
Episode 39794, Total Reward: -0.7000000000000005
Episode 39795, Total Reward: -1.7000000000000006
Episode 39796, Total Reward: -0.5000000000000002
Episode 39797, Total Reward: -0.7999999999999997
Episode 39798, Total Reward: -2.9
Episode 39799, Total Reward: -0.7
Episode 39800, Total Reward: -0.6999999999999995
Episode 39801, Total Reward: -1.7000000000000006
Episode 39802, Total Reward: -0.6000000000000003
E

Episode 39954, Total Reward: -0.8999999999999997
Episode 39955, Total Reward: -0.5999999999999998
Episode 39956, Total Reward: -0.7000000000000004
Episode 39957, Total Reward: -0.5999999999999995
Episode 39958, Total Reward: -0.4999999999999991
Episode 39959, Total Reward: -0.6000000000000003
Episode 39960, Total Reward: -3.6999999999999886
Episode 39961, Total Reward: -0.4999999999999995
Episode 39962, Total Reward: -0.6000000000000003
Episode 39963, Total Reward: -0.7000000000000004
Episode 39964, Total Reward: -1.7000000000000006
Episode 39965, Total Reward: -0.5999999999999995
Episode 39966, Total Reward: -1.9000000000000006
Episode 39967, Total Reward: -0.5000000000000003
Episode 39968, Total Reward: -1.8000000000000005
Episode 39969, Total Reward: -0.6000000000000003
Episode 39970, Total Reward: -1.8000000000000005
Episode 39971, Total Reward: -0.7999999999999998
Episode 39972, Total Reward: -0.6999999999999997
Episode 39973, Total Reward: -0.6999999999999995
Episode 39974, Total

In [2]:
import gym
import numpy as np
import torch

# Assuming the DQN, SortingEnv classes, and other necessary imports are already defined as before

# Initialize the environment
env = SortingEnv(list_length=LIST_LENGTH)
n_actions = env.action_space.n
state_size = env.observation_space.shape[0]

# Initialize the policy network and load the trained model
policy_net = DQN(env.observation_space, env.action_space)
policy_net.load_state_dict(torch.load("sorting_dqn_model.pth"))
policy_net.eval()  # Set the network to evaluation mode

def test_model(num_episodes=10):
    success_count = 0
    for i_episode in range(num_episodes):
        state = env.reset()
        state = torch.tensor(state, dtype=torch.float32)
        total_reward = 0
        done = False
        steps = 0
        print(f"\nEpisode {i_episode + 1}")
        print("Initial list:", env.list)
        while not done:
            # Select action using the trained policy (no exploration)
            with torch.no_grad():
                state_input = state.unsqueeze(0)  # Add batch dimension
                action = policy_net(state_input).argmax(dim=1).item()

            next_state, reward, done, _ = env.step(action)
            total_reward += reward
            steps += 1

#             print(f"Step {steps}:")
#             print(f"  Cursor position: {env.cursor}")
#             print(f"  Action taken: {action} ({['Move Left', 'Move Right', 'Compare', 'Swap'][action]})")
#             print(f"  Current list: {env.list}")
#             print(f"  Reward: {reward}")
#             print(f"  Done: {done}")

            state = torch.tensor(next_state, dtype=torch.float32)

        # Check if the list is sorted
        if np.all(env.list[:-1] <= env.list[1:]):
            print("Successfully sorted the list!")
            success_count += 1
        else:
            print("Failed to sort the list.")
        print(f"Total reward: {total_reward}")

    print(f"\nModel successfully sorted the list in {success_count} out of {num_episodes} episodes.")
    print(f"Accuracy = {success_count / num_episodes}")

# Run the test
test_model(num_episodes=2000)

observation_space size = 17

Episode 1
Initial list: [13  6 22  4 28  8  2]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 2
Initial list: [ 0 25 12  0 11 28  9]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 3
Initial list: [ 8 28 28 26 12 19  7]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 4
Initial list: [11 10  3 20 16 22 11]
Failed to sort the list.
Total reward: -0.7000000000000004

Episode 5
Initial list: [ 6 18 22  3  7 15  7]
Failed to sort the list.
Total reward: -0.8999999999999998

Episode 6
Initial list: [ 8 14 31  2 16  0 11]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 7
Initial list: [14 24 15 11  0 29 20]
Failed to sort the list.
Total reward: -0.7000000000000004

Episode 8
Initial list: [12 27 29 28 18  3 23]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 9
Initial list: [13  6 20  8 29 26 10]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 

Failed to sort the list.
Total reward: -0.6999999999999995

Episode 118
Initial list: [ 0  1 27  2 25 30 19]
Failed to sort the list.
Total reward: -0.6000000000000003

Episode 119
Initial list: [20 26 18 16 21 24  7]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 120
Initial list: [27 21 10 24 21 10  1]
Failed to sort the list.
Total reward: -0.6000000000000003

Episode 121
Initial list: [25 14 24  2  1 19 30]
Failed to sort the list.
Total reward: -0.6999999999999994

Episode 122
Initial list: [24  7 13  5 23 29  5]
Failed to sort the list.
Total reward: -0.8999999999999998

Episode 123
Initial list: [26 22  3 30 18 20  9]
Failed to sort the list.
Total reward: -0.5000000000000003

Episode 124
Initial list: [31  1 15 22  1 30  8]
Failed to sort the list.
Total reward: -0.7000000000000004

Episode 125
Initial list: [27  3  4 21 25  3 24]
Failed to sort the list.
Total reward: -0.6999999999999995

Episode 126
Initial list: [10 13  7 10  3  4 27]
Failed to sort the 

Failed to sort the list.
Total reward: -0.7000000000000004

Episode 212
Initial list: [21  6 11 29 17  8 30]
Failed to sort the list.
Total reward: -0.5999999999999995

Episode 213
Initial list: [ 3 30  3 20  9 21 31]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 214
Initial list: [ 2  6 11 26 22  3  2]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 215
Initial list: [19 24  6 28 17  1 13]
Failed to sort the list.
Total reward: -0.6000000000000003

Episode 216
Initial list: [27 30  1  6 16  5 10]
Failed to sort the list.
Total reward: -0.6999999999999995

Episode 217
Initial list: [ 7  2  7  8 12 21 27]
Successfully sorted the list!
Total reward: 10.1

Episode 218
Initial list: [18 29  6  8  0 17 12]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 219
Initial list: [16  6  1 24 23 25  5]
Failed to sort the list.
Total reward: -0.6999999999999994

Episode 220
Initial list: [11 25 26 23 16  9  2]
Failed to sort the list.
Tota

Failed to sort the list.
Total reward: -0.7000000000000004

Episode 335
Initial list: [ 9 21  7 13 17 26 13]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 336
Initial list: [25  0 14 30 19 17  5]
Failed to sort the list.
Total reward: -0.7000000000000004

Episode 337
Initial list: [31 11  7 18 29  0 29]
Failed to sort the list.
Total reward: -0.7999999999999996

Episode 338
Initial list: [31  3 29  4 11  0 13]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 339
Initial list: [21  5  1 22 30 11 20]
Failed to sort the list.
Total reward: -0.6000000000000003

Episode 340
Initial list: [14 24 12 15 15 13  2]
Failed to sort the list.
Total reward: -0.7000000000000004

Episode 341
Initial list: [ 3 18 25 14  3  5  6]
Failed to sort the list.
Total reward: -0.8999999999999998

Episode 342
Initial list: [ 5  8  2 20 12 21 19]
Failed to sort the list.
Total reward: -0.7999999999999996

Episode 343
Initial list: [19 18  8 29 28  2 24]
Failed to sort the 

Failed to sort the list.
Total reward: -0.9000000000000007

Episode 421
Initial list: [25 12  9  5 20 19 16]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 422
Initial list: [30  6 28 27 30 26  9]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 423
Initial list: [27 26 10 11 31 23  7]
Failed to sort the list.
Total reward: -0.7000000000000004

Episode 424
Initial list: [ 4 29 24 28 28 31 23]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 425
Initial list: [18 11 24  7 24 22 27]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 426
Initial list: [ 3 26  6 11 19 17  3]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 427
Initial list: [13 25 12 14 22  0  5]
Failed to sort the list.
Total reward: -0.7000000000000004

Episode 428
Initial list: [13 27 17 28  7 28  0]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 429
Initial list: [ 0  1 30  4 22  1  8]
Failed to sort the 

...

Failed to sort the list.
Total reward: -0.6999999999999997

Episode 1510
Initial list: [ 0 11 29 28 29 12 14]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1511
Initial list: [ 6  5 15 28 24  6 29]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 1512
Initial list: [ 9 28  2  4  3  6 31]
Failed to sort the list.
Total reward: -0.6999999999999995

Episode 1513
Initial list: [ 5 10  1 17 16 29  8]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 1514
Initial list: [ 0 29  2 10 28 18 20]
Failed to sort the list.
Total reward: -0.5999999999999995

Episode 1515
Initial list: [ 7  6 13 13 11 20 13]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1516
Initial list: [26  2 13 29 23 26  8]
Failed to sort the list.
Total reward: -0.6999999999999994

Episode 1517
Initial list: [23 22  1 11 11 16 12]
Failed to sort the list.
Total reward: -0.4999999999999993

Episode 1518
Initial list: [31 11 16 19  3 30 14]
Failed to 

Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1630
Initial list: [15 17 27 16  5 14 14]
Failed to sort the list.
Total reward: -0.8999999999999998

Episode 1631
Initial list: [10 11  9  5 30 25 23]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1632
Initial list: [ 8 14 12  3 30 19 18]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1633
Initial list: [26  1 25  2 28  7 17]
Failed to sort the list.
Total reward: -0.4999999999999994

Episode 1634
Initial list: [29 12 30  7  4  1 25]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 1635
Initial list: [0 3 8 6 6 1 3]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1636
Initial list: [25 13 27 27  8 28 25]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1637
Initial list: [13 27 10 30 19  0  3]
Failed to sort the list.
Total reward: -0.7999999999999998

Episode 1638
Initial list: [12 22 27 20 17  4  4]
Failed to sort th

Failed to sort the list.
Total reward: -0.5000000000000003

Episode 1720
Initial list: [29  5 21 17  5  3 25]
Failed to sort the list.
Total reward: -0.6999999999999995

Episode 1721
Initial list: [ 6  6  1 31 16 30 31]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1722
Initial list: [13  8 21 18  3  9 28]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1723
Initial list: [30 24 22 11  2  5 19]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 1724
Initial list: [14  6  8  2  5  1 21]
Failed to sort the list.
Total reward: -0.6999999999999994

Episode 1725
Initial list: [ 1 26 14 16 10 17 27]
Failed to sort the list.
Total reward: -0.7999999999999998

Episode 1726
Initial list: [23 14  0 20 22 29 23]
Failed to sort the list.
Total reward: -0.7999999999999996

Episode 1727
Initial list: [18 11  1 24  0  6 31]
Failed to sort the list.
Total reward: -0.6999999999999994

Episode 1728
Initial list: [23  7 15  8 14 21  1]
Failed to 

Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1846
Initial list: [16  4  1 24  6 16  4]
Failed to sort the list.
Total reward: -0.6999999999999994

Episode 1847
Initial list: [24 16 22  2 10  3  5]
Failed to sort the list.
Total reward: -0.8999999999999998

Episode 1848
Initial list: [ 0  1 13 14 18 15 11]
Failed to sort the list.
Total reward: -0.8999999999999998

Episode 1849
Initial list: [14 24 14  2  7 17  6]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 1850
Initial list: [19  9 21 11  5 17  0]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1851
Initial list: [14 23 16 12  5 27 17]
Failed to sort the list.
Total reward: -0.7000000000000004

Episode 1852
Initial list: [21 19 21 25 26 26  6]
Failed to sort the list.
Total reward: -0.8999999999999998

Episode 1853
Initial list: [22 31 13  1 17 20  7]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 1854
Initial list: [15 22  3 10  0  5 24]
Failed to 

Failed to sort the list.
Total reward: -0.5999999999999993

Episode 1936
Initial list: [20 10  8 17  7  0  3]
Failed to sort the list.
Total reward: -0.6000000000000003

Episode 1937
Initial list: [ 1 31 20  2  2 23  9]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1938
Initial list: [ 0 12  0 26 19  7  4]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1939
Initial list: [13 20  1 25  0  1 27]
Failed to sort the list.
Total reward: -0.6999999999999997

Episode 1940
Initial list: [ 3 19  9  5 20  9 27]
Failed to sort the list.
Total reward: -0.6000000000000003

Episode 1941
Initial list: [ 2  7  1 29 13 30 21]
Failed to sort the list.
Total reward: -0.7999999999999998

Episode 1942
Initial list: [ 7 10 17 22 12 27 30]
Failed to sort the list.
Total reward: -0.7999999999999998

Episode 1943
Initial list: [ 8 10  0  6 16  6 11]
Failed to sort the list.
Total reward: -0.8000000000000005

Episode 1944
Initial list: [23  3 11 18 18 15  1]
Failed to 

In [3]:
memory.memory[0].reward

AttributeError: 'BipartiteReplayMemory' object has no attribute 'memory'

In [ ]:
#torch.save(policy_net.state_dict(), 'datasets/dqn_4action_numeric/len6_maxlen16_3layer_hd128_20kepoch_accr8960_policynet.pth')